# Fashion MNIST

## Data Loading & Processing

In [62]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import tqdm

In [64]:
train_dataset = datasets.FashionMNIST(root="data", train=True, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)

In [66]:
images, _ = next(iter(train_loader))
mean = images.mean().item()
std = images.std().item()
print(mean, std)

0.28604060411453247 0.3530242443084717


In [67]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

In [68]:
train_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [69]:
class fashionMNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 32), 
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [70]:
%%time

model = fashionMNISTClassifier()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in tqdm.tqdm(range(10)):
    for X, y in train_loader:
        #forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|███████████████████████████████████████████| 10/10 [00:30<00:00,  3.05s/it]

CPU times: user 28.9 s, sys: 1.47 s, total: 30.4 s
Wall time: 30.5 s


## Evaluation 

In [76]:
model.eval()

correct = 0
total = 0

In [77]:
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        pred = outputs.argmax(dim=1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test accuracy percentage: {accuracy * 100:.2f}%")

Test accuracy percentage: 78.94%


## Convolutional Neural Network

In [81]:
class ConvolutionalClassifier(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        """
        Define the layers of the convolutional neural network.

        Parameters:
            in_channels: int
                The number of channels in the input image. For Fashion MNIST, this is 1 (grayscale images).
            num_classes: int
                The number of classes we want to predict, in our case 10.
        """
        super(ConvolutionalClassifier, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels = 128, kernel_size=[3, 3], padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )

        self.fc = nn.Sequential(
            nn.Linear(32 * 14 * 14, 128), #14 comes from 2x2 maxpool, the height and width are both halved
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)

        x = x.view(x.size(0), -1)   #batch size + automatic features by multiplying remaining dimensions [channels, height, width]
        x = self.fc(x)

        return x

In [83]:
%%time

model = ConvolutionalClassifier()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in tqdm.tqdm(range(10)):
    for X, y in train_loader:
        pred = model(X)
        loss = loss_fn(pred, y)

        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|██████████████████████████████████████████| 10/10 [38:15<00:00, 229.56s/it]

CPU times: user 58min 53s, sys: 4min 41s, total: 1h 3min 34s
Wall time: 38min 15s


In [87]:
model.eval()

correct = 0
total = 0

In [89]:
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        pred = outputs.argmax(dim=1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test accuracy percentage: {accuracy * 100:.2f}%")

Test accuracy percentage: 84.68%
